In [77]:
import pandas as pd
import numpy as np
import math
import matplotlib
import csv
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from matplotlib import pyplot
import pandas_profiling as pf
%matplotlib inline

In [81]:
missing_feature_map = {
    'Motor_Vehicle_License_Total' : {'op' : "ADD",
                                     'src_features' : ['Motor_Vehicle_License','Motor_Veh_Oper_License']
                                    }
    

    }

In [73]:
all_data="/Users/lohith/Desktop/NEU_Courses/ThirdSemester/CS6510-ASD/data/TheGovernmentFinanceDatabase_AllData/all.csv"

# Type_Code
STATE = 0
COUNTY = 1 
CITY = 2                                 
TOWNSHIP = 3 
SPECIAL_DIST = 4
SCHOOL_DIST = 5   
FED_GOVT = 6 
ADD=1
SUBSTRACT = -1

additionlist = []
substraction_list = []

In [74]:
def save_to_csv(d,file_name):
    """
        Writes contents of the given dict into a csv file
        Ex: d= {'Name':138,'Population':5,...}
        
        Args:
            d (dict)      : dictionary containing key value pair where key is the 
                            feature name and value is the number of missing zeros or nans
            filename(str) : output file name (absolute path)
    """
    with open(file_name, 'w') as fp:
        fwriter = csv.writer(fp)
        for val in d.items():
            fwriter.writerow(val)

def count_negatives(df):
    """
        Calculate the number of negative values for the features in the dataframe
        This applies only to numeric dtypes
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe
        Returns:
            dict : dictionary containing key value pair where key is the 
                   feature name and value is the number of missing zeros 
    """
    return (df.loc[:, df.dtypes != object] < 0).sum(axis=0).to_dict() 

def count_zeros(df):
    """
        Calculate the number of zeros for the features in the dataframe
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
        Returns:
            dict : dictionary containing key value pair where key is the 
                   feature name and value is the number of missing zeros 
    """
    return (df == 0).sum(axis=0).to_dict() 

def count_nans(df):
    """
        Calculate the number of nans for the features in the dataframe
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
        Returns:
            dict : dictionary containing key value pair where key as feature
            and value as the nan count for that feature
    """
    return (len(df) - df.count()).to_dict()
    

def drop_features(df, threshold=0.5):
    """
       Drop the features/columns from the dataframe based on the threshold.
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
            threshold(int [0,1]) : 
        Return:
            df (Pandas Dataframe)  : A pandas dataframe with columns with percentage of 
                                     null/nan > threshold 
            
    """
    total_instances=len(df)
    print(threshold,total_instances)
    
    to_drop=[]
    to_retain=[]
    zero_count = get_zero_count(df)
    nan_count = get_nan_count(df)
    for key, value in zero_count.items():
        total_invalid = value + nan_count.get(key)
        if total_invalid/total_instances >= threshold:
            to_drop.append(key)
        else:
            to_retain.append(key)
    return set(to_drop), set(to_retain)
        

In [79]:
class DataAnalyzer():
    
    def __init__(self,filepath):
        self.src_file = filepath
        self.df = pd.read_csv(self.src_file)
        
    def load_data(self):
        self.len = len(self.df)
        self.total_features = len(self.df.keys())
        self.features = self.df.keys()
        self.nans = (len(self.df) - self.df.count()).to_dict()
        self.zeros = (self.df == 0).sum(axis=0).to_dict() 
        self.negatives = (self.df.loc[:, self.df.dtypes != object] < 0).sum(axis=0).to_dict() 
        self.features_with_nan = self.df.columns[self.df.isna().any()].tolist()
        self.features_all_nan = [x for x,y in self.nans.items() if y == self.len ]
    
    def get_features(self, search_string):
        """
        Returns the list of features matching the search string.
        Args:
            search_string(str): search string.
        """
        return [x for x in self.features if search_string in x.lower()]
    
    def find_missing_years(self,feature):
        l = [feature,'Year4']
        _df = self.df[l].loc[self.df[feature] != self.df[feature]]
        n = len(_df)
        yl = _df['Year4'].unique().tolist()
        return "{}: {}({}%) missing values which are for years {}".format(feature , n, round(n * 100/total_instances,4),yl)
  
    def fill_missing(self, feature):
        if feature not in missing_feature_map:
            print("Feature not found")
            return
        
        if feature in self.df.keys():
            del self.df[feature]  
          
        op = missing_feature_map[feature]['op']
        features = missing_feature_map[feature]['src_features']
    
        if op == ADD:
            self.df['temp']  = self.df[features].sum(axis=1)  
        else:
            self.df['temp'] = self.df[features[0]] - self.df[features[1]]
            
        self.df.columns = self.df.columns.str.replace('temp',feature)  
    

In [80]:
#data = DataAnalyzer(all_data)

## Observations ##

      

**1. Feature has all 0 or nans ==> Drop it **

**2. County level data has valid values except for year 2016.**

**3. 6059 records have Name as "NOT AVAILABLE" **
     

  


## TODO ## 